In [ ]:
import os
from my_config import LANGFUSE_AUTH, OTEL_EXPORTER_OTLP_ENDPOINT, HF_TOKEN


os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = OTEL_EXPORTER_OTLP_ENDPOINT
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
# your Hugging Face token
os.environ["HF_TOKEN"] = HF_TOKEN

### Initialize the SmolagentsInstrumentor
Initialize the `SmolagentsInstrumentor` before your application code. Configure `tracer_provider` and add a span processor to export traces to Langfuse. `OTLPSpanExporter()` uses the endpoint and headers from the environment variables.

In [ ]:
from opentelemetry.sdk.trace import TracerProvider

from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter()))

SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

### Run Smolagent

In [ ]:
from huggingface_hub import login
login(token="")

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, HfApiModel
remote_url = "http://{}:11434"
model = LiteLLMModel(
    base_url=remote_url
    , model_id='ollama/qwen2.5-coder:7b'
    # , model_id='ollama/llama3.2:latest'
)


In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool, FinalAnswerTool
from my_tools import suggest_menu, catering_service_tool, SuperheroPartyThemeTool
# Alfred, the butler, preparing the menu for the party
agent = CodeAgent(
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        suggest_menu,
        catering_service_tool,
        SuperheroPartyThemeTool(),
        FinalAnswerTool()
        ],
    model=model,
    max_steps=10,
    verbosity_level=2
)


In [ ]:
agent.run("""
    Give me theh best list songs for a party at the Wayne's mansion. The party idea is a 'villain masquerade' theme.
    The output should be a list of song names and artists.
""")

## Tool Calling Agent

In [ ]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool

agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()]
    , model=model
    , max_steps=5
    , verbosity_level=2
)

agent.run("Search for the best music recommendations for a party at the Wayne's mansion.")